## Preparacion y publicacion de modelo ML en TabPy para consumir desde tableau Desktop

En este notebook crearemos un proceso de transformacion y predicción de precios de hosting de airbnb de Madrid, usando como entrada sus caracteristicas y modelos preentrenado previamente

In [1]:
# Hacer los imports
import warnings
warnings.filterwarnings('ignore')
from tabpy_client import Client
import joblib
import numpy as np
import re
import pandas as pd

Este proyecto ya tiene previamente entrenado dos modelos, un DecisionTreeRegressor y un RandomForestRegressor, por lo que unicamente los cargamos para consumirlos

In [2]:
#Cargamos los modelos preentrenados
tree_reg = joblib.load("tree_reg.pkl")
forest_reg = joblib.load( "forest_reg.pkl")

De igual manera todo el analisis exploratorio ya fue realizado previamente, por lo que solo retomamos la informacion relevante

In [3]:
variables_categoricas = ["property_type", "room_type", "bathrooms_text", "postal_code"]
variables_binarias = ["host_is_superhost", "host_identity_verified", "Bedroom comforts", "Hot water", "Kitchen", "TV",
                      "Cable TV", "Pocket wifi", "Changing table", "Patio or balcony", "Waterfront", "Bread maker",
                      "Full kitchen", "Heating", "Single level home", "Bathroom essentials", "Pets allowed",
                      "Game console", "Cooking basics", "Shampoo", "Bed linens", "First aid kit", "Piano", "Dishwasher",
                      "Extra pillows and blankets", "Iron", "Self check-in", "Garden or backyard", "Baby monitor",
                      "Beachfront", "Fire extinguisher", "Window guards", "Barbecue utensils", "Dishes and silverware",
                      "Ethernet connection", "Coffee maker", "BBQ grill", "Lake access", "Hot tub",
                      "Long term stays allowed", "Laptop-friendly workspace", "Paid parking on premises", "Stove",
                      "Smoke alarm", "Carbon monoxide alarm", "Microwave", "Refrigerator", "Free parking on premises",
                      "Lockbox", "Paid parking off premises", "Ski-in/Ski-out", "Shower gel", "Breakfast", "Dryer",
                      "Free street parking", "Air conditioning", "Stair gates", "Wifi", "Baking sheet",
                      "Lock on bedroom door", "Host greets you", "Suitable for events", "Cleaning before checkout",
                      "Hair dryer", "Children’s books and toys", "Room-darkening shades", "Outlet covers", "Keypad",
                      "Gym", "Fireplace guards", "Indoor fireplace", "Table corner guards", "Smoking allowed",
                      "Children’s dinnerware", "Essentials", "Washer", "Baby bath", "Luggage dropoff allowed",
                      "Elevator", "Smart lock", "Babysitter recommendations", "EV charger", "Building staff",
                      "Private living room", "Oven", "Beach essentials", "Crib", "Private entrance",
                      "Pack ’n Play/travel crib", "Pool", "Bathtub", "High chair", "Hangers"]
variables_numericas = ["host_listings_count", "number_of_reviews", "review_scores_rating", "accommodates", "bedrooms",
                       "beds", "antiguedad_host"]
variable_objetivo = "price_clean"
amenities_list = ['Bedroom comforts', 'Hot water', 'Kitchen', 'TV', 'Cable TV', 'Pocket wifi', 'Changing table',
                  'Patio or balcony', 'Waterfront', 'Bread maker', 'Full kitchen', 'Heating', 'Single level home',
                  'Bathroom essentials', 'Pets allowed', 'Game console', 'Cooking basics', 'Shampoo', 'Bed linens',
                  'First aid kit', 'Piano', 'Dishwasher', 'Extra pillows and blankets', 'Iron', 'Self check-in',
                  'Garden or backyard', 'Baby monitor', 'Beachfront', 'Fire extinguisher', 'Window guards',
                  'Barbecue utensils', 'Dishes and silverware', 'Ethernet connection', 'Coffee maker', 'BBQ grill',
                  'Lake access', 'Hot tub', 'Long term stays allowed', 'Laptop-friendly workspace',
                  'Paid parking on premises', 'Stove', 'Smoke alarm', 'Carbon monoxide alarm', 'Microwave',
                  'Refrigerator', 'Free parking on premises', 'Lockbox', 'Paid parking off premises', 'Ski-in/Ski-out',
                  'Shower gel', 'Breakfast', 'Dryer', 'Free street parking', 'Air conditioning', 'Stair gates', 'Wifi',
                  'Baking sheet', 'Lock on bedroom door', 'Host greets you', 'Suitable for events',
                  'Cleaning before checkout', 'Hair dryer', 'Children’s books and toys', 'Room-darkening shades',
                  'Outlet covers', 'Keypad', 'Gym', 'Fireplace guards', 'Indoor fireplace', 'Table corner guards',
                  'Smoking allowed', 'Children’s dinnerware', 'Essentials', 'Washer', 'Baby bath',
                  'Luggage dropoff allowed', 'Elevator', 'Smart lock', 'Babysitter recommendations', 'EV charger',
                  'Building staff', 'Private living room', 'Oven', 'Beach essentials', 'Crib', 'Private entrance',
                  'Pack ’n Play/travel crib', 'Pool', 'Bathtub', 'High chair', 'Hangers']

def getCleanValue(o):
    remove = re.compile(r'[^\d\.]+')
    s = "0.00"
    try:
        s = remove.sub(" ", str(o)).strip()
    except:
        s = "0.00"
    f=0.00
    try:
        f= np.float64(s)
    except:
        f = np.float64(0.00)
    return f

Creamos el pipeline de transformacion y normalizacion de los features que espera el modelo

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=True)),
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(missing_values=0.00, strategy="mean")),
    ('std_scaler', StandardScaler()),
])

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, variables_numericas),
    ("cat", cat_pipeline, variables_categoricas),
    ("bool", StandardScaler(), variables_binarias)
])

Recargamos el dataset con el que se preparo y entreno el modelo, de manera que nuestro pipeline haga el fit de los features a transformar

In [5]:
#Importamos un dataset ya preparado para hacer fit de los pipelines de tranformacion a utilizar
import pandas as pd

df = pd.read_csv("train_prepared.csv", index_col=0)
df = df.drop(variable_objetivo, axis=1)
for x in variables_categoricas:
    df[x] = df[x].astype(str)
for x in variables_binarias:
    df[x] = df[x].fillna(0)
    df[x] = df[x].astype(np.int8)
for x in variables_numericas:
    df[x] = df[x].apply(lambda c: getCleanValue(c))
    df[x] = df[x].astype(float)
    
full_pipeline.fit_transform(df)

array([[-3.15683088e-01,  3.46136059e-01,  3.14701121e-01, ...,
        -2.19562851e-01, -2.43148090e-01,  4.65670421e-01],
       [-2.98614475e-01, -7.25677624e-01, -1.51276671e+00, ...,
        -2.19562851e-01, -2.43148090e-01,  4.65670421e-01],
       [-2.55166739e-02,  0.00000000e+00, -1.73132532e-15, ...,
        -2.19562851e-01, -2.43148090e-01,  4.65670421e-01],
       ...,
       [ 9.39636142e-02,  0.00000000e+00, -1.73132532e-15, ...,
        -2.19562851e-01,  4.11271995e+00,  4.65670421e-01],
       [-1.62065575e-01, -6.94153692e-01, -6.59948389e-01, ...,
        -2.19562851e-01, -2.43148090e-01,  4.65670421e-01],
       [-3.15683088e-01,  0.00000000e+00, -1.73132532e-15, ...,
         4.55450452e+00, -2.43148090e-01,  4.65670421e-01]])

Finalmente creamos una funcion que recibira como parametros los features necesarios y en base a estos devuelve el valor predicho por el modelo de Random Forest

In [6]:
def hostPricePredictor(_arg1, _arg2, _arg3, _arg4, _arg5, _arg6, _arg7, _arg8, _arg9, _arg10, _arg11, _arg12, _arg13, _arg14):
        
    #Transformamos a diccionario para cargar en un DataFrame    
    tab_datos = {"host_is_superhost": _arg1
                ,"host_listings_count": _arg2
                ,"host_identity_verified": _arg3
                ,"property_type": _arg4
                ,"room_type": _arg5
                ,"accommodates": _arg6
                ,"bathrooms_text": _arg7
                ,"bedrooms": _arg8
                ,"beds": _arg9
                ,"number_of_reviews": _arg10
                ,"review_scores_rating": _arg11
                ,"antiguedad_host": _arg12
                ,"postal_code": _arg13
                ,"amenities": _arg14
                }

    df = pd.DataFrame.from_dict(data=tab_datos, orient='columns')
    
    #Preparamos los datos
    for a in amenities_list:
        df[a] = df.amenities.loc[np.logical_not(df.amenities.isna())].apply(lambda c: 1 if a in c else 0)
    df = df.drop(["amenities"], axis=1)
    
    for x in variables_categoricas:
        df[x] = df[x].astype(str)
    for x in variables_binarias:
        df[x] = df[x].fillna(0)
        df[x] = df[x].astype(np.int8)
    for x in variables_numericas:
        df[x] = df[x].apply(lambda c: getCleanValue(c))
        df[x] = df[x].astype(float)
    #Transformamos segun el pipeline
    listing_prepared = full_pipeline.transform(df)
    
    #Ejecutamos la prediccion
    tree_predictions = tree_reg.predict(listing_prepared)
    forest_predictions = forest_reg.predict(listing_prepared)
    
    #Devolvemos los resultados
    return [price for price in forest_predictions]


In [7]:
#Probamos la funcion
hostPricePredictor([1], [1], [1], ["Private room"], ["Private room"], [2], ["1 bath"], [2], [2], [50], [90], [2], ["28053"], ['["Air conditioning", "Washer", "Kitchen", "Private entrance", "Wifi", "Heating", "TV", "Essentials"]'])

[49.81513249706778]

### Publicacion en tableau

Una vez ya tenemos nuestra función creada y funcionando, nos conectamos a TabPy en nuestro servidor local y hacemos deploy de la función para que pueda ser consumida posteriormente en tableau desktop

In [8]:
cliente = Client("http://localhost:9004")
cliente.remove("hostPricePredictor")
cliente.deploy("hostPricePredictor", hostPricePredictor, "Calcula el precio del hosting segun los parametros recibidos", override = True)

Ya con esto nos vamos a tableau desktop y usamos la funcion publicada